In [1]:
import jax 
import jax.numpy as jnp
import pennylane as qml
from jax import custom_jvp
from qbc_ipe_jax_probs import qbc_ipe_algorithm
from jax import make_jaxpr
jax.config.update("jax_enable_x64", True)

W =  [-1.6193685   1.28386154 -1.13687517 -0.4885566 ]
b =  0.21635686180382116
x =  [ 0.52  1.12  0.77 -1.07]
rho =  4.318032192757666 rho_exact =  0.24321485484507657
x =  [ 0.88 -1.08  0.15  0.26]
rho =  -2.1776358645962466 rho_exact =  -3.10917082283958
x =  [ 0.52  0.06 -1.3   0.51]
rho =  3.462068811534412 rho_exact =  0.46373379634507633
x =  [ 0.74 -2.49  1.39  0.62]
rho =  -3.574409954031523 rho_exact =  -6.27830960772888
x =  [ 0.52  1.12  0.77 -1.07]
rho =  4.318032192757666 rho_exact =  0.24321485484507657
rho =  Traced<ShapedArray(float64[])>with<BatchTrace(level=1/1)> with
  val = Array([1.50185622, 0.39575532, 1.14588359, 0.5243316 ], dtype=float64)
  batch_dim = 0 rho_exact =  Traced<ShapedArray(float64[])>with<BatchTrace(level=1/1)> with
  val = Array([ 0.52000018,  1.12000006,  0.77000006, -1.06999994], dtype=float64)
  batch_dim = 0
rho =  2.8144563596514726e-08 rho_exact =  -1.930437541950596e-07
primal_out =  4.318032192757666 primal_out_exact =  0.2432149859000001

In [2]:
key = jax.random.PRNGKey(0)

# Build a toy dataset.
inputs = jnp.array([[0.52, 1.12,  0.77, -1.07],
                   [0.88, -1.08, 0.15, 0.26],
                   [0.52, 0.06, -1.30, 0.51],
                   [0.74, -2.49, 1.39, 0.62]])
targets = jnp.array([True, True, False, True])

# Initialize random model coefficients
key, W_key, b_key = jax.random.split(key, 3)
W = jax.random.normal(W_key, (4,))
b = jax.random.normal(b_key, ())
print("W = ", W)
print("b = ", b)

W =  [-1.6193685   1.28386154 -1.13687517 -0.4885566 ]
b =  0.21635686180382116


In [3]:
@custom_jvp
def my_inner_prod(x, y):
    assert len(x) == len(y)
    n = len(x)
    res = 0.0
    for i in range(n):
        res += x[i] * y[i]
    return res

@my_inner_prod.defjvp
def my_inner_prod_jvp(primals, tangents):
    primal_out = my_inner_prod(*primals)
    tangent_out = my_inner_prod(primals[1], tangents[0]) + my_inner_prod(primals[0], tangents[1])
    return primal_out, tangent_out

@custom_jvp
def qbc_ipe_inner_prod(x, y):
    result, _, _, _, _ = qbc_ipe_algorithm(x, y)
    # theta = 2 * jnp.pi * j / 2**10
    # f = jnp.sin(theta / 2) ** 2

    # result = -(1.0 - 2.0 * f)
    # x_norm = jnp.linalg.norm(x)
    # y_norm = jnp.linalg.norm(y)
    # result = -(1.0 - 2.0 * result)
    print("x = ", x)
    print("y = ", y)
    print("result = ", result)
    return result

@qbc_ipe_inner_prod.defjvp
def qbc_ipe_inner_prod_jvp(primals, tangents):
    # primal_out = qbc_ipe_inner_prod(*primals)
    # tangent_out = qbc_ipe_inner_prod(primals[1], tangents[0]) + qbc_ipe_inner_prod(primals[0], tangents[1])
    print("primals = ", primals)
    print("primals[0] = ", primals[0])
    print("primals[1] = ", primals[1])
    vector_1_dot, vector_2_dot = tangents
    print("tangents[0] = ", tangents[0])
    print("tangents[1] = ", tangents[1])
    print("vector_1_dot = ", vector_1_dot)
    print("vector_2_dot = ", vector_2_dot)
    primal_out, _, _, _, _ = qbc_ipe_algorithm(primals[0], primals[1])
    print("primal_out =", primal_out)
    tangent_out_1, _, _, _, _ = qbc_ipe_algorithm(primals[1], tangents[0]) 
    tangent_out_2, _, _, _, _ = qbc_ipe_algorithm(primals[0], tangents[1])
    
    
    tangent_out = tangent_out_1
    print("tangent_out =", tangent_out)

    return primal_out, tangent_out

In [4]:
def sigmoid(x):
    return (jnp.tanh(x / 2.0) + 1.0) / 2.0

def predict(W, b, inputs):
    res = []
    for x in inputs:
        print("x = ", x)
        z = qbc_ipe_inner_prod(W, x) + b
        f_z = sigmoid(z)
        res.append(z)
    return jnp.array(res)

def loss(W, b):
    preds = predict(W, b, inputs)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -jnp.sum(jnp.log(label_probs))

In [5]:
b_grad = jax.jacfwd(loss, argnums=1)(W, b)
print(b_grad)

x =  [ 0.52  1.12  0.77 -1.07]
rho =  4.318032200309959 rho_exact =  0.24321485541210025


TypeError: iteration over a 0-d array

In [ ]:
W_grad = jax.jacfwd(loss, argnums=0)(W, b)
print(W_grad)

In [ ]:
make_jaxpr(jax.jacrev(loss, argnums=0))

[-0.16965583 -0.8774644  -1.4901346 ] -0.29227245

[ 0.10766962  0.6651212  -0.00334581] -0.29227245

In [ ]:
jax.jvp(loss, (W, b), (W_grad, b_grad))

(Array(3.0519388, dtype=float32), Array(3.1046512, dtype=float32))

(Array(3.0519388, dtype=float32), Array(0.53941333, dtype=float32))

In [ ]:
print(inputs)

In [ ]:
result, _, _, j, probs = qbc_ipe_algorithm(jnp.array([0.5, 1.0]), jnp.array([-1.0, 1.0]))

In [ ]:
print(result)
print(j)

In [ ]:
import math
theta = 2 * jnp.pi * j / float(2**10)
print(type(theta))
f = math.sin(theta / 2.0)

rho = -(1.0 - 2.0 * f) * jnp.linalg.norm(jnp.array([0.5, 1.0])) * jnp.linalg.norm(jnp.array([-1.0, 1.0]))
print(f)
print(rho)

In [6]:
import pennylane as qml


In [7]:
@jax.jit
def func():
    dev = qml.device("default.qubit.jax", wires=4)

    # @jax.jit
    # def had():

    n_wires = range(0, 2)
    a_wires = range(2, 3)
    t_wires = range(3, 4)
    x = jnp.array([-1.0, 2.0])
    x /= jnp.linalg.norm(x)
    tmp = jnp.zeros((2**2, 2**2 - 1))
    A_x = jnp.concatenate(
        (jnp.pad(x, pad_width=(0, 2**2 - len(x))).reshape(-1, 1), tmp),
        axis=1,
    )

    Q_x, _ = jnp.linalg.qr(A_x, mode="full")
    for i in range(2**2):
        Q_x.at[:, i].set(Q_x[:, i] / jnp.linalg.norm(Q_x[:, i]))

    u_x = Q_x.T
    u_x *= jnp.sign(u_x[0, 0]) * jnp.sign(x[0])


    y = jnp.array([-1.0, 2.0])
    y /= jnp.linalg.norm(y)
    A_y = jnp.concatenate(
        (jnp.pad(y, pad_width=(0, 2**2 - len(x))).reshape(-1, 1), tmp),
        axis=1,
    )
    Q_y, _ = jnp.linalg.qr(A_y, mode="full")
    for i in range(2**2):
        Q_y.at[:, i].set(Q_y[:, i] / jnp.linalg.norm(Q_y[:, i]))

    u_y = Q_y.T
    u_y *= jnp.sign(u_y[0, 0]) * jnp.sign(y[0])

    # @jax.jit
    def U_x():
        qml.QubitUnitary(u_x, wires=n_wires)

    # @jax.jit
    def U_y():
        qml.QubitUnitary(u_x, wires=n_wires)

    # @jax.jit
    def A_operator():
        qml.Hadamard(wires=a_wires)
        qml.ctrl(U_x, control=a_wires, control_values=0)()
        qml.ctrl(U_y, control=a_wires, control_values=1)()
        qml.Hadamard(wires=a_wires)

    # @jax.jit
    @qml.qnode(dev, interface="jax")
    def circuit():
        qml.ctrl(A_operator, control=t_wires, control_values=0)()
        A_operator()
        A_operator()
        return qml.state()

    return circuit()

In [8]:
print(func())
print(func())

[-0.4472136 +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.89442719+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
[-0.4472136 +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.89442719+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
